In [ ]:
'''
This notebook contains the code to merge all PRICES data across the years based on sections.
There are datasets for 2012, 2013 and 2014

'''

In [ ]:
'''
The following code block will merge all "Basic Information about union council Respondents" files across the years.
* First, we are going to read the respective files and store them as data frames
* Next, we are going to define column mappings that I have already figured out via manual methods
'''

In [15]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd

# Store excel file locations to variables (change it as per your path to file)

path_2013=r"C:\Users\warra\Desktop\Freelance\data\data\PriceMerge\1. Merging by Parts\3. Section 1 Part 2 Basic Information about Union Council Level Respondents\2013_s1p2_prices.csv"
path_2014=r"C:\Users\warra\Desktop\Freelance\data\data\PriceMerge\1. Merging by Parts\3. Section 1 Part 2 Basic Information about Union Council Level Respondents\2014_s1p2_prices.csv"

# Read csv files

df_2013 = pd.read_csv(path_2013)
df_2014 = pd.read_csv(path_2014)


In [16]:
'''
This code block will standardize column names across the years to avoid discrepancies during the merging process.
For example, the cover data for 2013 is noth the same as in 2014 data, but they have the same data under their differently named variables. Hence, we decide to rename such columns beforehand
We will add the updated name to the mapping dictionaries instead of the original names.
'''

# Rename columns in df_2013
df_2013.rename(columns={
    's1p2_q7' : 'UCLR_STI',
    's1p2_q8' : 'UCLR_ETI',
    's1p2_q9' : 'UCLR_DOI' 
}, inplace=True)

# df_2014 doesn't need renaming as it is the reference

In [17]:
# Updated mappings
mapping_2013 = [
    'cid', 'round', 's1p2_q1', None, None, None, None, None, None, None, None, 'UCLR_STI', 'UCLR_ETI', 'UCLR_DOI'
]

mapping_2014 = [
    'cid', 'round', 's1p2_q1', 's1p2_q6_c', 's1p2_q7_hh', 's1p2_q7_mm',
    's1p2_q8_hh', 's1p2_q8_mm', 's1p2_q9_dd', 's1p2_q9_mm', 's1p2_q9_yy',
    None, None, None
]


In [18]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [19]:
import pandas as pd
import numpy as np

def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [20]:
# Usage with dataframes and mappings
dfs = [df_2013, df_2014]
mappings = [mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['cid', 'round', 's1_part_no', 's1p2_q1', 'UCLR_STI', 'UCLR_ETI', 'UCLR_DOI']
Appending data for column cid
Appending data for column round
Appending data for column s1p2_q1
Appending data for column UCLR_STI
Appending data for column UCLR_ETI
Appending data for column UCLR_DOI
Processing DataFrame with columns: ['Unnamed: 0', 'cid', 'round', 's1p2_q1', 's1p2_q6_c', 's1p2_q7_hh', 's1p2_q7_mm', 's1p2_q8_hh', 's1p2_q8_mm', 's1p2_q9_dd', 's1p2_q9_mm', 's1p2_q9_yy']
Appending data for column cid
Appending data for column round
Appending data for column s1p2_q1
Appending data for column s1p2_q6_c
Appending data for column s1p2_q7_hh
Appending data for column s1p2_q7_mm
Appending data for column s1p2_q8_hh
Appending data for column s1p2_q8_mm
Appending data for column s1p2_q9_dd
Appending data for column s1p2_q9_mm
Appending data for column s1p2_q9_yy


In [21]:
# Rename columns for the merged file (if needed)
rename_mapping = {
    'cid': 'CID',
    'round': 'Survey_Round',
    's1p2_q1': 'UCLR_Shop_Type',
    's1p2_q6_c': 'UCLR_Caste',
    's1p2_q7_hh': 'UCLR_STI_hrs',
    's1p2_q7_mm': 'UCLR_STI_min',
    's1p2_q8_hh': 'UCLR_ETI_hrs',
    's1p2_q8_mm': 'UCLR_ETI_min',
    's1p2_q9_dd': 'UCLR_DOI_d',
    's1p2_q9_mm': 'UCLR_DOI_m',
    's1p2_q9_yy': 'UCLR_DOI_y'
}

merged_df.rename(columns=rename_mapping, inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('merged_Prices_Basic-Information-about-Union-Council-Level-Respondents.csv', index=False)
